In [1]:
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from ticker_download_predict_upload import DownloadPredictUpload
from sklearn.metrics import root_mean_squared_error

### Get pieces of the main script

These will be extended in this notebook.

In [2]:
dpu = DownloadPredictUpload()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


### Get the ticker histories

In [3]:
long_df_filename = os.path.join("input", "Tickers 2025-03-25.csv")
long_df = pd.read_csv(long_df_filename)
wide_df = dpu.pivot_ticker_close_wide(long_df)
wide_df


ticker,AAPL,AMZN,GOOG,MSFT,NVDA,TSLA
2025-01-27 17:00:00,229.86,235.42,193.77,434.56,118.42,397.15
2025-01-28 17:00:00,238.26,238.15,197.07,447.20,128.99,398.09
2025-01-29 17:00:00,239.36,237.07,197.18,442.33,123.70,389.10
2025-01-30 17:00:00,237.59,234.64,202.63,414.99,124.65,400.28
2025-01-31 17:00:00,236.00,237.68,205.60,415.06,120.07,404.60
2025-02-03 17:00:00,228.01,237.42,202.64,410.92,116.66,383.68
2025-02-04 17:00:00,232.80,242.06,207.71,412.37,118.65,392.21
2025-02-05 17:00:00,232.47,236.17,193.30,413.29,124.83,378.17
2025-02-06 17:00:00,233.22,238.83,193.31,415.82,128.68,374.32
2025-02-07 17:00:00,227.63,229.15,187.14,409.75,129.84,361.62


### Train a Holt-Winters model

Without seasonality

In [4]:
def walk_forward(task):
    y_trues = task["ts"]
    train_len = task["train_len"]
    y_preds = np.array([np.nan for _ in range(len(y_trues))])
    print(len(y_trues))
    print(len(y_preds))
    for i in range(train_len, len(y_trues)):
        train = y_trues[i - train_len : i]
        model = ExponentialSmoothing(train, trend="add", seasonal=None, use_boxcox=0.0)
        fit = model.fit()
        y_pred = fit.forecast(steps=1)
        y_preds[i] = y_pred
    df = pd.DataFrame({"y_true": y_trues, "y_pred": y_preds}).dropna()
    print(train_len)
    rmse = root_mean_squared_error(df["y_true"], df["y_pred"])
    return rmse, df

### Test on NVDA

In [5]:
task = {
    "ts": wide_df["NVDA"],
    "train_len": 20,
}
result_rmse, result_df = walk_forward(task)
print(result_rmse)
result_df

40
40
20
5.546266651133573


/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/alicia/miniconda3/envs/fuzzy-system-research/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/var/folders/_f/zmxbx_t93v96nklchvbj6j2r0000gn/T/ipykernel_14323/557621560.py:12: FutureWarning: Calling float on a single eleme

,y_true,y_pred
2025-02-25 17:00:00,126.63,130.934449
2025-02-26 17:00:00,131.28,126.508118
2025-02-27 17:00:00,120.15,131.693169
2025-02-28 17:00:00,124.92,119.923082
2025-03-03 17:00:00,114.06,120.942651
2025-03-04 17:00:00,115.99,113.520208
2025-03-05 17:00:00,117.30,111.843677
2025-03-06 17:00:00,110.57,112.862436
2025-03-07 17:00:00,112.69,109.560901
2025-03-10 17:00:00,106.98,109.717606
